In [14]:
from utils.segmentation import YOLOSegmentation
import cv2
import numpy as np
import rospy
from cv_bridge import CvBridge, CvBridgeError
from contact_grasp.srv import segmentationSrv, segmentationSrvResponse
from utils.camera_utils import RealCamera, RealCameraROS
from utils.visualisation_utils import depth2pc
from utils.transform_utils import regularize_pc
from utils.ROS_utils import generate_grasps_client, format_pointcloud_msg, run_action
import open3d as o3d
import time

In [15]:
def display_inlier_outlier(cloud, ind):
    inlier_cloud = cloud.select_by_index(ind)
    outlier_cloud = cloud.select_by_index(ind, invert=True)

    print("Showing outliers (red) and inliers (gray): ")
    outlier_cloud.paint_uniform_color([1, 0, 0])
    # inlier_cloud.paint_uniform_color([0.8, 0.8, 0.8])
    o3d.visualization.draw_geometries([inlier_cloud, outlier_cloud])

In [16]:
camera_connexion = "ROS"
if camera_connexion == "ROS":
    camera = RealCameraROS()
    intrinsic, distortion = camera.getIntrinsic()
elif camera_connexion == "pyWrapper":
    camera = RealCamera()
    camera.start()
    #retrieve image and depth
    for i in range(15):
        rgb, depth_image, depth_scale = camera.get_rgb_depth()

    rgb, depth_image, depth_scale = camera.get_rgb_depth()
    intrinsic, distortion = camera.getIntrinsic()
else:
    raise Exception("Please choose a valid camera connexion method: ROS or pyWrapper")


Camera topic found


In [17]:
cv2.imshow("rgb", rgb)
cv2.waitKey(0)
cv2.destroyAllWindows()

QObject::moveToThread: Current thread (0x564e643d4330) is not the object's thread (0x564e5fda7620).
Cannot move to target thread (0x564e643d4330)

QObject::moveToThread: Current thread (0x564e643d4330) is not the object's thread (0x564e5fda7620).
Cannot move to target thread (0x564e643d4330)

QObject::moveToThread: Current thread (0x564e643d4330) is not the object's thread (0x564e5fda7620).
Cannot move to target thread (0x564e643d4330)

QObject::moveToThread: Current thread (0x564e643d4330) is not the object's thread (0x564e5fda7620).
Cannot move to target thread (0x564e643d4330)

QObject::moveToThread: Current thread (0x564e643d4330) is not the object's thread (0x564e5fda7620).
Cannot move to target thread (0x564e643d4330)

QObject::moveToThread: Current thread (0x564e643d4330) is not the object's thread (0x564e5fda7620).
Cannot move to target thread (0x564e643d4330)

QObject::moveToThread: Current thread (0x564e643d4330) is not the object's thread (0x564e5fda7620).
Cannot move to tar

In [18]:
rgb, depth_image, depth_scale = camera.get_rgb_depth()
depth = depth_image * depth_scale

bridge = CvBridge()

pc_fused, pc_colors_fused = depth2pc(depth, intrinsic, rgb)

In [19]:
pc_fused, pc_colors_fused = regularize_pc(pc_fused, pc_colors_fused, downsampling_method="voxel", voxel_size=0.005, outlier_filtering_method="radius", radius_param_arg=[25, 0.015])

Downsampling time: 0.6162905693054199s, Filtering time: 0.0724186897277832s


In [20]:
pc2_msg = format_pointcloud_msg(pc_fused, pc_colors_fused)
bgr = cv2.cvtColor(rgb, cv2.COLOR_RGB2BGR)
bgr_msg = bridge.cv2_to_imgmsg(bgr, encoding="bgr8")

In [28]:
orn, pos, opening, score, detected, detected_with_collision = generate_grasps_client(pc2_msg, bgr_msg)

calling service
